# Budowanie z modelami Mistral

## Wprowadzenie

W tej lekcji omówimy:
- Przegląd różnych modeli Mistral
- Zrozumienie zastosowań i scenariuszy dla każdego modelu
- Przykłady kodu pokazujące unikalne cechy każdego modelu


## Modele Mistral

W tej lekcji przyjrzymy się trzem różnym modelom Mistral: **Mistral Large**, **Mistral Small** oraz **Mistral Nemo**.

Każdy z tych modeli jest dostępny za darmo na rynku modeli Github. Kod w tym notatniku będzie wykorzystywał te modele do uruchamiania kodu. Więcej informacji o korzystaniu z modeli Github do [prototypowania z modelami AI](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 to obecnie flagowy model firmy Mistral, zaprojektowany z myślą o zastosowaniach biznesowych.

Model ten stanowi ulepszenie w stosunku do oryginalnego Mistral Large, oferując:
- Większe okno kontekstu – 128k w porównaniu do 32k
- Lepszą wydajność w zadaniach matematycznych i programistycznych – średnia dokładność 76,9% w porównaniu do 60,4%
- Zwiększoną wydajność wielojęzyczną – obsługiwane języki to: angielski, francuski, niemiecki, hiszpański, włoski, portugalski, niderlandzki, rosyjski, chiński, japoński, koreański, arabski i hindi.

Dzięki tym funkcjom Mistral Large doskonale sprawdza się w:
- *Retrieval Augmented Generation (RAG)* – dzięki większemu oknu kontekstu
- *Function Calling* – model ten posiada natywną obsługę wywołań funkcji, co umożliwia integrację z zewnętrznymi narzędziami i API. Wywołania te mogą być wykonywane równolegle lub sekwencyjnie, jedno po drugim.
- *Generowanie kodu* – model świetnie radzi sobie z generowaniem kodu w językach Python, Java, TypeScript oraz C++.


W tym przykładzie używamy Mistral Large 2 do uruchomienia wzorca RAG na dokumencie tekstowym. Pytanie jest napisane po koreańsku i dotyczy działalności autora przed studiami.

Wykorzystuje model Cohere Embeddings do tworzenia osadzeń zarówno dokumentu tekstowego, jak i pytania. W tym przykładzie jako magazyn wektorów używany jest pakiet faiss dla Pythona.

Prompt wysyłany do modelu Mistral zawiera zarówno pytania, jak i odnalezione fragmenty podobne do pytania. Model następnie generuje odpowiedź w języku naturalnym.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small to kolejny model z rodziny Mistral, należący do kategorii premier/enterprise. Jak sama nazwa wskazuje, jest to Mały Model Językowy (SLM). Korzyści z używania Mistral Small to:
- Oszczędność kosztów w porównaniu do modeli Mistral LLM, takich jak Mistral Large i NeMo – spadek ceny o 80%
- Niska latencja – szybsza odpowiedź w porównaniu do LLM-ów Mistral
- Elastyczność – można wdrażać w różnych środowiskach, z mniejszymi wymaganiami dotyczącymi zasobów

Mistral Small sprawdza się w:
- Zadaniach tekstowych, takich jak podsumowywanie, analiza sentymentu i tłumaczenie
- Aplikacjach, w których często pojawiają się zapytania, dzięki swojej opłacalności
- Zadaniach związanych z kodem, gdzie liczy się niska latencja, takich jak przegląd i sugestie dotyczące kodu


## Porównanie Mistral Small i Mistral Large

Aby zobaczyć różnice w opóźnieniu między Mistral Small a Large, uruchom poniższe komórki.

Powinieneś zauważyć różnicę w czasie odpowiedzi wynoszącą od 3 do 5 sekund. Zwróć też uwagę na długość i styl odpowiedzi na ten sam prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

W porównaniu do dwóch pozostałych modeli omawianych w tej lekcji, Mistral NeMo jest jedynym darmowym modelem z licencją Apache2.

Jest postrzegany jako ulepszona wersja wcześniejszego otwartoźródłowego LLM od Mistral, czyli Mistral 7B.

Inne cechy modelu NeMo to:

- *Bardziej wydajna tokenizacja:* Model ten korzysta z tokenizera Tekken zamiast popularniejszego tiktoken. Dzięki temu lepiej radzi sobie z różnymi językami i kodem.

- *Dostępność do finetuningu:* Model bazowy można dostosowywać do własnych potrzeb. Daje to większą elastyczność w przypadkach, gdy wymagane jest dostrojenie.

- *Natywne wywoływanie funkcji* – Podobnie jak Mistral Large, ten model został wytrenowany do wywoływania funkcji. To czyni go wyjątkowym, ponieważ jest jednym z pierwszych otwartoźródłowych modeli z tą funkcjonalnością.


## Mistral NeMo

W porównaniu do dwóch pozostałych modeli omawianych w tej lekcji, Mistral NeMo jest jedynym darmowym modelem z licencją Apache2.

Jest postrzegany jako ulepszona wersja wcześniejszego otwartoźródłowego LLM od Mistral, czyli Mistral 7B.

Inne cechy modelu NeMo to:

- *Bardziej wydajna tokenizacja:* Model ten korzysta z tokenizera Tekken zamiast popularniejszego tiktoken. Dzięki temu lepiej radzi sobie z różnymi językami i kodem.

- *Dostępność do dostrajania (finetuning):* Model bazowy jest dostępny do dalszego dostrajania. Daje to większą elastyczność w przypadkach, gdy potrzebne jest dostosowanie modelu do konkretnych zastosowań.

- *Natywne wywoływanie funkcji* – Podobnie jak Mistral Large, ten model został wytrenowany do wywoływania funkcji. To czyni go wyjątkowym, ponieważ jest jednym z pierwszych otwartoźródłowych modeli z tą funkcjonalnością.


### Porównanie tokenizatorów

W tym przykładzie przyjrzymy się, jak Mistral NeMo radzi sobie z tokenizacją w porównaniu do Mistral Large.

Oba przykłady korzystają z tego samego promptu, ale powinieneś zauważyć, że NeMo zwraca mniej tokenów niż Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Nauka nie kończy się tutaj, kontynuuj swoją podróż

Po ukończeniu tej lekcji zajrzyj do naszej [kolekcji materiałów o Generative AI](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), aby dalej rozwijać swoją wiedzę na temat Generative AI!



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony przy użyciu usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Dokładamy wszelkich starań, aby tłumaczenie było poprawne, jednak należy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być traktowany jako źródło nadrzędne. W przypadku informacji o kluczowym znaczeniu zalecane jest skorzystanie z profesjonalnych usług tłumaczeniowych. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
